In [2]:
import torch
import torch.nn as nn

# Simple model with torch.flip
class FlipModel(nn.Module):
    def __init__(self):
        super(FlipModel, self).__init__()

    def forward(self, x):
        # Flip along the width dimension
        x_flipped = torch.flip(x, [3])  # Flip along the last dimension (Width)
        return x_flipped

# Create model and dummy input
model = FlipModel()
dummy_input = torch.randn(1, 3, 224, 224)  # (Batch, Channels, Height, Width)

# Perform inference in PyTorch
output_torch = model(dummy_input)
print("PyTorch output shape:", output_torch.shape)


PyTorch output shape: torch.Size([1, 3, 224, 224])


In [5]:
import onnx

# Export the model
onnx_filename = "flip_model.onnx"

# Exporting the model
torch.onnx.export(
    model,
    dummy_input,
    onnx_filename,
    export_params=True,                # store the trained parameter weights
    opset_version=16,                  # ONNX opset version
    input_names=["input"],             # Input names
    output_names=["output"],           # Output names
    dynamic_axes={
        "input": {0: "batch_size"},    # dynamic batch size
        "output": {0: "batch_size"}
    }
)

print(f"Model exported to {onnx_filename}")


Model exported to flip_model.onnx


In [8]:
import onnxruntime as ort
import numpy as np

# Load the exported ONNX model
session = ort.InferenceSession(onnx_filename)

# Prepare the input tensor for ONNX Runtime
onnx_input = dummy_input.detach().numpy()
onnx_input_dict = {"input": onnx_input}

# Run inference
onnx_output = session.run(None, onnx_input_dict)[0]

# Verify results
print("ONNX output shape:", onnx_output.shape)

# Compare ONNX and PyTorch outputs
torch_output_np = output_torch.detach().numpy()
diff = np.abs(onnx_output - torch_output_np)

print("\nDifference between PyTorch and ONNX outputs:", np.max(diff))
print("Inference completed successfully!")


ONNX output shape: (1, 3, 224, 224)

Difference between PyTorch and ONNX outputs: 0.0
Inference completed successfully!
